In [2]:
from dolfin import *
# from msh2xdmf import import_mesh_from_xdmf, msh2xdmf


In [ ]:

# MAKE CFD
def sim_flow(u_0, nu, cp, k, p_0, T_0, fileName, Le, He, We):
    
    # LOAD MESH
    
    mesh, boundaries, association_table = import_mesh_from_xdmf(prefix=fileName, dim=3)
    
    # Build function space
    P2 = VectorElement('Lagrange', mesh.ufl_cell() , 2)
    P1 = FiniteElement('Lagrange', mesh.ufl_cell(), 1)
    element = MixedElement([P2, P1, P1])
    W = FunctionSpace(mesh, element)
    #define test functions
    (v, q, s) = TestFunctions(W)
    #split functions
    upT = Function(W)
    (u, p, T) = split(upT)

    n = FacetNormal(mesh)
    
    # Define boundary conditions
    # FLOW
    # Define inflow profile from Shah & London 1976 (velocity profile in a rectangular duct)
    alpha = min(We/He, He/We) # <1
    mv = 1.7 + 0.5 * alpha**-1.4
    if alpha<=1./3.:
        nv = 2
    else:
        nv = 2 + 0.3 * (alpha - 1./3.)
    Umax = u_0 * (mv+1)/mv * (nv+1)/nv
    print("alpha, m, n, Umax = ", alpha, mv, nv, Umax)
    inflow_profile = ('0', '0', 'Umax * (1. - pow(abs(x[0]/H*2), n)) * (1. -  pow(abs(x[1]/W*2), m))')
    inflow_profile = Expression(inflow_profile, Umax=Constant(Umax), H=Constant(He), W=Constant(We), m=Constant(mv), n=Constant(nv), degree=2)
    bcu_inflow = DirichletBC(W.sub(0), inflow_profile, boundaries, association_table["inlet"])
    bcu_noslip = DirichletBC(W.sub(0), Constant((0, 0 ,0 )), boundaries, association_table["noslip"])
    bcu_outflow = DirichletBC(W.sub(1), Constant(p_0), boundaries, association_table["outlet"])
    bcu = [bcu_inflow, bcu_noslip, bcu_outflow]
    
    # ENERGY
    bcT_inflow = DirichletBC(W.sub(2), Constant(T_0), boundaries, association_table["inlet"])
    bcT_noslip = DirichletBC(W.sub(2), Constant(T_0+10.), boundaries, association_table["noslip"])
    bcT = [bcT_inflow, bcT_noslip]
    
    bcs = bcu + bcT
    
    # DEFINE VARIATIONAL FORM
     
    F1 = (inner(grad(u)*u, v)*dx +                 # Momentum advection term
        nu*inner(grad(u), grad(v))*dx -          # Momentum diffusion term
        inner(p, div(v))*dx +                    # Pressure term
        div(u)*q*dx                            # Divergence term
    ) 
    F2 = (cp*inner(dot(grad(T), u), s)*dx - # Energy advection term
        k*inner(grad(T), grad(s))*dx # Energy diffusion term
    )
    F = F1 + F2
    
    # Create VTK files for visualization output
    vtkfile_u = File('results/u.pvd')
    vtkfile_p = File('results/p.pvd')
    vtkfile_T = File('results/T.pvd')